# Gen AI

This page provides the details on the mlflow gen AI module.

In [1]:
import mlflow
import logging
from mlflow.tracking import MlflowClient
from IPython.display import clear_output

import logging
from langchain_ollama import ChatOllama

logging.basicConfig(level=logging.WARNING)

!rm -f /tmp/llm_mlflow.db
mlflow.set_registry_uri("sqlite:////tmp/llm_mlflow.db")
mlflow.set_tracking_uri("sqlite:////tmp/llm_mlflow.db")
chat = ChatOllama(model="llama3.2:1b", temperature=0)

## Trace

Each invocation to the LLM based pipeline is called "trace".

Trace cosits of:

- [**Trace info**](https://mlflow.org/docs/3.3.0/genai/tracing/concepts/trace/#traceinfo-metadata-and-context): general information about the trace primarly used for models ordering and selection.
- [**Trace data**](https://mlflow.org/docs/3.3.0/genai/tracing/concepts/trace/#tracedata-container-of-spans): detailed information about the pipline run. Consists of spans.

---

The following cell runs the experiment that produces the traces we will consider later.

In [2]:
mlflow.langchain.autolog()
ans = chat.invoke("Hello how are you?")
clear_output()

With `mlflow.search_traces` you can get the traces registered in your mlflows as `pandas.DataFrame`.

In [3]:
traces = mlflow.search_traces()
traces

,trace_id,trace,client_request_id,state,request_time,execution_duration,request,response,trace_metadata,tags,spans,assessments
0,tr-06f1ef26ee7df2eee94b73b1af90c415,"{""info"": {""trace_id"": ""tr-06f1ef26ee7df2eee94b...",None,TraceState.OK,1764150102464,3665,"[[{'content': 'Hello how are you?', 'additiona...","{'generations': [[{'text': ""I'm doing well, th...","{'mlflow.user': 'user', 'mlflow.source.name': ...",{'mlflow.artifactLocation': '/home/user/Docume...,"[{'trace_id': 'BvHvJu598u7pS3Oxr5DEFQ==', 'spa...",[]


The following cell loads a specific trace.

In [4]:
trace_obj = mlflow.get_trace(traces["trace_id"].iloc[0])

The following cell shows loads the `info` of the trace.

In [5]:
trace_obj.info

TraceInfo(trace_id='tr-06f1ef26ee7df2eee94b73b1af90c415', trace_location=TraceLocation(type=<TraceLocationType.MLFLOW_EXPERIMENT: 'MLFLOW_EXPERIMENT'>, mlflow_experiment=MlflowExperimentLocation(experiment_id='0'), inference_table=None), request_time=1764150102464, state=<TraceState.OK: 'OK'>, request_preview='[[{"content": "Hello how are you?", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": null}]]', response_preview='{"generations": [[{"text": "I\'m doing well, thank you for asking. Is there anything I can help you with or would you like to talk about something in particular?", "generation_info": {"model": "llama3.2:1b", "created_at": "2025-11-26T09:41:46.127363708Z", "done": true, "done_reason": "stop", "total_duration": 3658687704, "load_duration": 1875642829, "prompt_eval_count": 30, "prompt_eval_duration": 394490334, "eval_count": 30, "eval_duration": 1350268885, "model_name": "llama3.2:1b", "model_provider": "ollama"}, "type": "ChatGenerat

This cell displays the `data` from the trace.

In [6]:
trace_obj.data

TraceData(spans=[Span(name='ChatOllama', trace_id='tr-06f1ef26ee7df2eee94b73b1af90c415', span_id='c9eca46a0a240231', parent_id=None)])

## Custom tracing

To build a custom tracing framework you have to specify specify the spans creation logic and which information have to be logged to them:

- The `mlflow.tracing` decorator to make a span each time the function is called.
- The `mlflow.start_span` context manager to track everything that happens inside the span.

**Note**: Some spans can be nested to the other spans.

Check more in the:

- [Manual Tracing](https://mlflow.org/docs/latest/genai/tracing/app-instrumentation/manual-tracing/) guide.
- [mlflow.trace](https://mlflow.org/docs/latest/api_reference/python_api/mlflow.html#mlflow.trace) decorator API.

---

The following cell creates the traced `some_tracing` function which simply wraps the input in special wrapped text.

In [3]:
ans = mlflow.set_experiment("custom_tracing")

2025/11/26 15:08:42 INFO mlflow.tracking.fluent: Experiment with name 'custom_tracing' does not exist. Creating a new experiment.


The following cell uses context manager syntax to create the span.

In [7]:
with mlflow.start_span() as span:
    span.set_inputs("Input request")
    span.attributes["hello"] = "new_value"
    span.set_outputs("Span outputs")
clear_output()

An alternative approach is to wrap the function with `mlflow.trace` decorator, so that its invocation is tracked as a span. 

In [5]:
@mlflow.trace
def some_tracing(inp: str) -> str:
    return f"<extra information>{inp}<the data>"

some_tracing("hello")
clear_output()

The following cell shows the kind of span that would be obtained from the corresponding trace.

In [6]:
trace_id = mlflow.search_traces()["trace_id"].iloc[0]
trace = mlflow.get_trace(trace_id=trace_id)

span = trace.data.spans[0]
print(f"{span.inputs} -> {span.outputs}")

{'inp': 'hello'} -> <extra information>hello<the data>


### Customizing spans

You can attach the following to each span:

- `name`.
- `span_type`: specified as value from predefined `mlflow.entities.SpanType`.
- `attrributes`: key-value pairs.

---

The following cell creates the span as the context, and invokes there some functions that are decorated as spans. Each span have different setup.

In [10]:
from mlflow.entities import SpanType

@mlflow.trace(
    name="my_span",
    span_type=SpanType.LLM,
    attributes={"attr1": "value1"}
)
def llm_span(input: str) -> str:
    return "some value"

@mlflow.trace(span_type=SpanType.RERANKER)
def reranker_span(input: str) -> str:
    return "output of rerunker span"

with mlflow.start_span(span_type="custom span", name="my_cool_span"):
    llm_span("input")
    reranker_span("reranker span input")

## Evaluation

The MLFlow provides the tool for evaluation is the `mlflow.getia.evaluate` function. You have to provide:

- `data`: dataset that would be used for evaluation. 
- `predict_fn`: function that implements the model. 
- `scorers`: list of evaluation objects, custom or provided by mlflow.

Check more on [evaluation and monitoring](https://mlflow.org/docs/latest/genai/eval-monitor/).

---

The following cell performs the evaluation.

In [ ]:
from mlflow.genai import scorer
from langchain.messages import AIMessage

mlflow.set_experiment("evaluation_test")

eval_dataset = [
    {
        "inputs": {"question": "What is the Scotland's national animal?"},
        "expectations": {"expected_response": "Unicorn"},
    },
    {
        "inputs": {"question": "Who was the first person to build an airplane?"},
        "expectations": {"expected_response": "Wright Brothers"},
    },
    {
        "inputs": {"question": "Who wrote Romeo and Juliet?"},
        "expectations": {"expected_response": "William Shakespeare"},
    },
]

def predict_fn(question: str) -> str:
    return chat.invoke(question)


@scorer
def some_check(outputs: AIMessage) -> bool:
    return len(outputs.content) < 10

mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=predict_fn,
    scorers=[some_check]
)
clear_output()

After that, you should see the corresponding interface in the MLFlow UI that describes the outputs of the evaluation.